# DABstep Benchmark Qwen2.5-Coder-32B Baseline

This notebook will guide you though submitting a Qwen2.5-Coder-32B baseline to the DABstep leaderboard.

* Live 🤗 Leaderboard: https://huggingface.co/spaces/adyen/DABstep
* Benchmark 🤗 Dataset: https://huggingface.co/datasets/adyen/DABstep
* LLM Agent Framework by 🤗: https://github.com/huggingface/smolagents/tree/main



## Environment Setup

We need to setup:
* **HuggingFace Token:** In order to make free API calls to HuggingFace Inference API you must have a HF account, the API verifies this by checking your account's token. This token will not be used for anything else.
* **Benchmark context files:** In order to solve the benchmark tasks the agent will need to reference documentation and analyze data which is spread out across multiple files, just like a real Data Analyst would.

### HuggingFace Token Setup

In [164]:
# %pip install smolagents==1.6.0 datasets==3.2.0
# # we install also the evaluation logic from the DABstep leaderboard, so we can locally evaluate without having to make a submission
# %pip install git+https://git@hf.co/spaces/adyen/DABstep.git@main

In [165]:
import time
import os
import json
import re
import datasets
import pandas as pd
from smolagents import CodeAgent
from smolagents.agents import ActionStep
from smolagents.models import OpenAIServerModel
from huggingface_hub import hf_hub_download
from datetime import datetime

# Load API keys from secrets
openrouter_key = None
nanogpt_key = None

# Load OpenRouter API key
openrouter_path = os.path.abspath(os.path.join(os.getcwd(), "..", "secrets", "openrouter_credentials.txt"))
if os.path.exists(openrouter_path):
    with open(openrouter_path, "r") as f:
        content = f.read()
    # Extract API key - look for pattern like sk-or-v1-...
    key_match = re.search(r"sk-or-v1-[A-Za-z0-9]+", content)
    if key_match:
        openrouter_key = key_match.group(0)
        print("✓ Loaded OpenRouter API key from secrets")
    else:
        print("⚠️ Warning: No valid OpenRouter key found in", openrouter_path)
else:
    print("⚠️ Warning: OpenRouter credentials file not found at", openrouter_path)

# Load NanoGPT API key
nanogpt_path = os.path.abspath(os.path.join(os.getcwd(), "..", "secrets", "nanogpt_credentials.txt"))
if os.path.exists(nanogpt_path):
    with open(nanogpt_path, "r") as f:
        content = f.read()
    # Extract API key - look for pattern like api_key = 'xxx' or just the key itself
    key_match = re.search(r"api_key\s*=\s*['\"]([^'\"]+)['\"]", content)
    if key_match:
        nanogpt_key = key_match.group(1)
        print("✓ Loaded NanoGPT API key from secrets")
    else:
        # Try to find a UUID pattern (NanoGPT format)
        key_match = re.search(r"[0-9a-f]{8}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{12}", content)
        if key_match:
            nanogpt_key = key_match.group(0)
            print("✓ Loaded NanoGPT API key from secrets")
        else:
            print("⚠️ Warning: No valid NanoGPT key found in", nanogpt_path)
else:
    print("⚠️ Warning: NanoGPT credentials file not found at", nanogpt_path)

# Try to import DABstep utilities
try:
    from dabstep_benchmark.utils import evaluate
    DABSTEP_AVAILABLE = True
except ImportError:
    print("Warning: DABstep benchmark utilities not available.")
    DABSTEP_AVAILABLE = False
    def evaluate(*args, **kwargs):
        raise RuntimeError("DABstep utilities not installed.")

notebook_start_time = time.time()
print(f'Right now is {datetime.now().isoformat()}')


✓ Loaded OpenRouter API key from secrets
✓ Loaded NanoGPT API key from secrets
Right now is 2025-11-12T19:51:32.629505


#### Download context files
First we download the context files from the [Benchmark's Dataset](https://huggingface.co/datasets/adyen/DABstep) so that our agent can access them.


In [166]:
CONTEXT_FILENAMES = [
    "data/context/acquirer_countries.csv",
    "data/context/payments-readme.md",
    "data/context/payments.csv",
    "data/context/merchant_category_codes.csv",
    "data/context/fees.json",
    "data/context/merchant_data.json",
    "data/context/manual.md",
]
DATA_DIR = "/tmp/DABstep-data"
for filename in CONTEXT_FILENAMES:
    hf_hub_download(
        repo_id="adyen/DABstep",
        repo_type="dataset",
        filename=filename,
        local_dir=DATA_DIR,
        force_download=True
    )

CONTEXT_FILENAMES = [f"{DATA_DIR}/{filename}" for filename in CONTEXT_FILENAMES]

for file in CONTEXT_FILENAMES:
    if os.path.exists(file):
        print(f"{file} exists.")
    else:
        print(f"{file} does not exist.")

/tmp/DABstep-data/data/context/acquirer_countries.csv exists.
/tmp/DABstep-data/data/context/payments-readme.md exists.
/tmp/DABstep-data/data/context/payments.csv exists.
/tmp/DABstep-data/data/context/merchant_category_codes.csv exists.
/tmp/DABstep-data/data/context/fees.json exists.
/tmp/DABstep-data/data/context/merchant_data.json exists.
/tmp/DABstep-data/data/context/manual.md exists.


## Agent

Here we will setup a simple zero-shot prompt for the agent. It has two parts, the general prompt and then a quick outline of which files are available.

In [167]:
# API Provider Selection
PROVIDER = "nanogpt"  # Options: "openrouter" or "nanogpt"

# Available OpenRouter models
MODELS_OPENROUTER = {
    "qwen2.5": "qwen/qwen-2.5-coder-32b-instruct",
    "qwen-coder-480b": "qwen/qwen3-coder",
    "deepseek-v3.1": "deepseek/deepseek-chat-v3.1",
    "deepseek-v3-terminus": "deepseek/deepseek-v3.1-terminus",
    "deepseek-v3.2": "deepseek/deepseek-v3.2-exp",
    "kimi-k2-0905": "moonshotai/kimi-k2-0905",
    "kimi-k2-thinking": "moonshotai/kimi-k2-thinking",
    "glm-4.5-air": "z-ai/glm-4.5-air",
    "glm-4.5": "z-ai/glm-4.5",
    "glm-4.6": "z-ai/glm-4.6",
    "gpt-oss-20b": "openai/gpt-oss-20b",
    "gpt-oss-120b": "openai/gpt-oss-120b",
    "gpt-5-mini": "openai/gpt-5-mini",
    "gpt-5-nano": "openai/gpt-5-nano",
    "grok-code-fast1": "x-ai/grok-code-fast-1",
    "grok-4-fast": "x-ai/grok-4-fast",
    "minimax-m2": "minimax/minimax-m2",
}

# Available NanoGPT models (verified from API)
MODELS_NANOGPT = {
    "qwen2.5-32b": "Qwen/Qwen2.5-Coder-32B-Instruct",
    "qwen3-coder": "qwen/qwen3-coder",
    "deepseek-v3": "deepseek-v3-0324",
    "deepseek-v3.1": "deepseek-ai/DeepSeek-V3.1",
    "deepseek-r1": "deepseek-r1",
    "gemma_3_27b": "google/gemma-3-27b-it",
    "deepseek_v3_1_reasoner": "deepseek/deepseek-reasoner",
    "kimi_k2_thinking": "moonshotai/kimi-k2-thinking",
    "kimi_k2_0905": "moonshotai/kimi-k2-0905", # Specific snapshot  
    "glm_4_5": "zhipu/glm-4.5",
    "glm_4_6": "zhipu/glm-4.6",
    "qwen_3_30b_moe": "qwen/qwen3-30b-a3b",
    "qwen_3_235b_moe": "qwen/qwen3-235b-a22b", # beloware verified ones
    "gpt_oss_20b": "openai/gpt-oss-20b",
    "gpt_oss_120b": "openai/gpt-oss-120b",
    'kimi_k2_instruct_fast': 'kimi-k2-instruct-fast',
    'deepseek_v3_0324': 'deepseek-v3-0324',
    'moonshotai_kimi_k2_instruct_0905': 'moonshotai/Kimi-K2-Instruct-0905',
    'moonshotai_kimi_k2_thinking': 'moonshotai/kimi-k2-thinking',   
    'qwen3_30b_a3b_instruct_2507': 'qwen3-30b-a3b-instruct-2507',
    'qwen3_coder_30b_a3b_instruct': 'qwen3-coder-30b-a3b-instruct',
    'qwen3_vl_235b_a22b_thinking': 'qwen3-vl-235b-a22b-thinking',
    'qwen_qwen2_5_coder_32b_instruct': 'Qwen/Qwen2.5-Coder-32B-Instruct',
    'qwen_qwen3_235b_a22b_instruct_2507': 'Qwen/Qwen3-235B-A22B-Instruct-2507',
    'qwen_qwen3_235b_a22b_thinking_2507': 'Qwen/Qwen3-235B-A22B-Thinking-2507',
    'qwen_qwen3_max': 'qwen/qwen3-max',
    'qwen_qwen3_next_80b_a3b_instruct': 'Qwen/Qwen3-Next-80B-A3B-Instruct',
    'z_ai_glm_4_6': 'z-ai/glm-4.6',
    'zai_org_glm_4_5_air': 'zai-org/GLM-4.5-Air',
    'zai_org_glm_4_5_fp8': 'zai-org/GLM-4.5-FP8',
}

# Select provider and model
if PROVIDER == "openrouter":
    if not openrouter_key:
        raise ValueError("OpenRouter API key not found. Please check secrets/openrouter_credentials.txt")
    MODEL_KEY = "glm_4_5"
    MODEL_ID = MODELS_OPENROUTER[MODEL_KEY]
    model = OpenAIServerModel(
        model_id=MODEL_ID,
        api_base="https://openrouter.ai/api/v1",
        api_key=openrouter_key
    )
    print(f"✓ Using OpenRouter model: {MODEL_ID}")
    
elif PROVIDER == "nanogpt":
    if not nanogpt_key:
        raise ValueError("NanoGPT API key not found. Please check secrets/nanogpt_credentials.txt")
    MODEL_KEY = "z_ai_glm_4_6"
    MODEL_ID = MODELS_NANOGPT[MODEL_KEY]
    model = OpenAIServerModel(
        model_id=MODEL_ID,
        api_base="https://nano-gpt.com/api/v1",
        api_key=nanogpt_key
    )
    print(f"✓ Using NanoGPT model: {MODEL_ID}")
    
else:
    raise ValueError(f"Unknown provider: {PROVIDER}. Choose 'openrouter' or 'nanogpt'")


✓ Using NanoGPT model: z-ai/glm-4.6


## 3.4 Testing Agent

In [168]:
MAX_STEPS = 7

agent = CodeAgent(
    tools=[],
    model=model,
    additional_authorized_imports=["numpy", "pandas", "json", "csv", "os", "glob", "markdown"],
    max_steps=MAX_STEPS,
    verbosity_level=3,
)
# give agent power to open files
agent.python_executor.static_tools.update({"open": open})

In [169]:
PROMPT = """You are an expert data analyst and you will answer factoid questions by loading and referencing the files/documents listed below.
You have these files available:
{context_files}
Don't forget to reference any documentation in the data dir before answering a question.

Here is the question you need to answer:
{question}

Here are the guidelines you must follow when answering the question above:
{guidelines}
"""
question = "What are the unique set of merchants in the payments data?"
guidelines = "Answer with a comma separated list"

PROMPT = PROMPT.format(
    context_files=CONTEXT_FILENAMES,
    question=question,
    guidelines=guidelines
)

agent_start_time = time.time()
answer = agent.run(PROMPT)
agent_end_time = time.time()


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an expert data analyst and you will answer factoid questions by loading and referencing the             │
│ files/documents listed below.                                                                                   │
│ You have these files available:                                                                                 │
│ ['/tmp/DABstep-data/data/context/acquirer_countries.csv', '/tmp/DABstep-data/data/context/payments-readme.md',  │
│ '/tmp/DABstep-data/data/context/payments.csv', '/tmp/DABstep-data/data/context/merchant_category_codes.csv',    │
│ '/tmp/DABstep-data/data/context/fees.json', '/tmp/DABstep-data/data/context/merchant_data.json',                │
│ '/tmp/DABstep-data/data/context/manual.md']                                                                     │
│ Don't forget to reference any documentation in the data dir before answering a question.                        │
│                                                                                                                 │
│ Here is the question you need to answer:                                                                        │
│ What are the unique set of merchants in the payments data?                                                      │
│                                                                                                                 │
│ Here are the guidelines you must follow when answering the question above:                                      │
│ Answer with a comma separated list                                                                              │
│                                                                                                                 │
╰─ OpenAIServerModel - z-ai/glm-4.6 ──────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                   
Thought: I need to find the unique set of merchants in the payments data. First, let me examine the available files
to understand the data structure. I should start by looking at the payments-readme.md file to understand the       
payments data structure, then examine the payments.csv file to extract the unique merchants.                       
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
# First, let's read the readme file to understand the data structure                                               
with open('/tmp/DABstep-data/data/context/payments-readme.md', 'r') as f:                                          
    readme_content = f.read()                                                                                      
print(readme_content)                                                                                              
```                                                                                                                

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  # First, let's read the readme file to understand the data structure                                             
  with open('/tmp/DABstep-data/data/context/payments-readme.md', 'r') as f:                                        
      readme_content = f.read()                                                                                    
  print(readme_content)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
This is documentation for the payments.csv dataset


- **Description**: Synthetic dataset of payment transactions processed by the Payments Processor.
- **Columns**:
  - `psp_reference`: Unique payment identifier (ID).
  - `merchant`: Merchant name (Categorical), eg Starbucks or Netflix*.
  - `card_scheme`: Card Scheme used (Categorical) - *[MasterCard, Visa, Amex, Other]*.
  - `year`: Payment initiation year (Numeric).
  - `hour_of_day`: Payment initiation hour (Numeric).
  - `minute_of_hour`: Payment initiation minute (Numeric).
  - `day_of_year`: Day of the year of payment initiation (Numeric).
  - `is_credit`: Credit or Debit card indicator (Categorical).
  - `eur_amount`: Payment amount in euro (Numeric).
  - `ip_country`: The country the shopper was in at time of transaction (determined by IP address) (Categorical) - 
*[SE, NL, LU, IT, BE, FR, GR, ES]*.
  - `issuing_country`: Card-issuing country (Categorical) - *[SE, NL, LU, IT, BE, FR, GR, ES]*.
  - `device_type`: Device type used (Categorical) - *[Windows, Linux, MacOS, iOS, Android, Other]*.
  - `ip_address`: Hashed shopper's IP (ID).
  - `email_address`: Hashed shopper's email (ID).
  - `card_number`: Hashed card number (ID).
  - `shopper_interaction`: Payment method (Categorical) - *[Ecommerce, POS]*. POS means an in-person or in-store 
transaction.
  - `card_bin`: Bank Identification Number (ID).
  - `has_fraudulent_dispute`: Indicator of fraudulent dispute from issuing bank (Boolean).
  - `is_refused_by_adyen`: Adyen refusal indicator (Boolean).
  - `aci`: Authorization Characteristics Indicator (Categorical).
  - `acquirer_country`: The location (country) of the acquiring bank (Categorical) - *[SE, NL, LU, IT, BE, FR, GR, 
ES]*.

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                   
Thought: I have successfully extracted the unique merchants from the payments data. There are 5 unique merchants:  
Belles_cookbook_store, Crossfit_Hanna, Golfclub_Baron_Friso, Martinis_Fine_Steakhouse, and Rafa_AI. Now I need to  
provide the answer as a comma-separated list as requested.                                                         
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
# Create comma-separated list of unique merchants                                                                  
merchant_list = ", ".join(sorted(unique_merchants))                                                                
final_answer(merchant_list)                                                                                        
```[{'id': 'call_4', 'type': 'function', 'function', 'arguments': '# Create comma-separated list of unique         
merchants\nmerchant_list = ", ".join(sorted(unique_merchants))\nfinal_answer(merchant_list)'}}]                    

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  # Create comma-separated list of unique merchants                                                                
  merchant_list = ", ".join(sorted(unique_merchants))                                                              
  final_answer(merchant_list)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Belles_cookbook_store, Crossfit_Hanna, Golfclub_Baron_Friso, Martinis_Fine_Steakhouse, Rafa_AI

[Step 2: Duration 6.69 seconds| Input tokens: 8,190 | Output tokens: 414]

In [170]:
notebook_end_time = time.time()

# Extract step count from agent monitor
total_steps = len(agent.monitor.step_durations)
print(f"✓ Agent execution completed in {total_steps} steps")


✓ Agent execution completed in 3 steps


In [171]:
# You can inspect the steps taken by the agent by doing this
def clean_reasoning_trace(trace: list) -> list:
  for step in trace:
      # Remove memory from logs to make them more compact.
      if hasattr(step, "memory"):
          step.memory = None
      if isinstance(step, ActionStep):
          step.agent_memory = None
  return trace

for step in clean_reasoning_trace(agent.logs):
    print(step)


SystemPromptStep(system_prompt='You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.\nTo do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.\nTo solve the task, you must plan forward to proceed in a series of steps, in a cycle of \'Thought:\', \'Code:\', and \'Observation:\' sequences.\n\nAt each step, in the \'Thought:\' sequence, you should first explain your reasoning towards solving the task and the tools that you want to use.\nThen in the \'Code:\' sequence, you should write the code in simple Python. The code sequence must end with \'<end_code>\' sequence.\nDuring each intermediate step, you can use \'print()\' to save whatever important information you will then need.\nThese print outputs will then appear in the \'Observation:\' field, which will be available as input for the next step.\nIn the end you have to return a final answer using t

In [172]:
print(f"Notebook runtime: {notebook_end_time - notebook_start_time:.2f} seconds")
print(f'Right now is {datetime.now().isoformat()}')


Notebook runtime: 22.46 seconds
Right now is 2025-11-12T19:51:55.098000


In [173]:
# Test Summary - Steps, Timing, Answer, and Persistence
import csv
import glob

print("\n" + "="*80)
print("TEST SUMMARY")
print("="*80)

# Print provider and model info
print(f"\n🔌 Provider: {PROVIDER}")
print(f"📊 Model: {MODEL_ID}")
print(f"❓ Question: {question}")

# Calculate metrics
print(f"\n📈 Total Steps: {total_steps}")

try:
    total_time = agent_end_time - agent_start_time
    print(f"\n⏱️  Total Agent Time: {total_time:.2f}s")
    print(f"   Notebook Runtime: {notebook_end_time - notebook_start_time:.2f}s")
except NameError:
    print("\n⚠️  Agent timing information not available")

# Print final answer
print(f"\n💬 Final Answer:")
print(f"{answer}")

# Print token info
print(f"\n🔤 Tokens:")
print(f"   Input Tokens: {total_input_tokens}")
print(f"   Output Tokens: {total_output_tokens}")

# Prepare results directory - save to repo/results/toy_results with timestamped filename
# Navigate from notebooks directory to repo root
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
results_dir = os.path.join(repo_root, "results", "toy_results")
os.makedirs(results_dir, exist_ok=True)

# Use timestamped filename to keep old results separate
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
results_file = os.path.join(results_dir, f"results_with_open_fix_{timestamp}.csv")

# Prepare result row
result_row = {
    "timestamp": datetime.now().isoformat(),
    "provider": PROVIDER,
    "model": MODEL_ID,
    "question": question,
    "total_steps": total_steps,
    "total_agent_time_s": round(total_time, 2),
    "notebook_runtime_s": round(notebook_end_time - notebook_start_time, 2),
    "input_tokens": total_input_tokens,
    "output_tokens": total_output_tokens,
    "answer": str(answer)
}

# Save to CSV
file_exists = os.path.isfile(results_file)
with open(results_file, 'a', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=result_row.keys())
    if not file_exists:
        writer.writeheader()
    writer.writerow(result_row)

print(f"\n✅ Results saved to: {results_file}")

# Display all results from ALL runs (combine all timestamped files)
print("\n" + "="*80)
print("ALL RESULTS (ALL RUNS)")
print("="*80)

# Find all results files matching the pattern
all_results_files = sorted(glob.glob(os.path.join(results_dir, "results_with_open_fix_*.csv")))

if all_results_files:
    # Read and combine all results files
    all_results = []
    for results_file_path in all_results_files:
        df = pd.read_csv(results_file_path)
        all_results.append(df)
    
    results_df = pd.concat(all_results, ignore_index=True)
    print("\n")
    print(results_df.to_string(index=False))
else:
    print("No results files found")
    results_df = pd.DataFrame()

print("\n" + "="*80)



TEST SUMMARY

🔌 Provider: nanogpt
📊 Model: z-ai/glm-4.6
❓ Question: What are the unique set of merchants in the payments data?

📈 Total Steps: 3

⏱️  Total Agent Time: 19.80s
   Notebook Runtime: 22.46s

💬 Final Answer:
Belles_cookbook_store, Crossfit_Hanna, Golfclub_Baron_Friso, Martinis_Fine_Steakhouse, Rafa_AI

🔤 Tokens:
   Input Tokens: 7554
   Output Tokens: 366

✅ Results saved to: /home/mykola/repos/dabstep_test/results/toy_results/results_with_open_fix_20251112_195155.csv

ALL RESULTS (ALL RUNS)


                 timestamp                            model                                                   question  total_steps  total_agent_time_s  notebook_runtime_s  input_tokens  output_tokens                                                                                                                                                                                                                                                                                                

In [174]:
# Calculate total tokens from agent's monitor
# IMPORTANT: Use agent.monitor.get_total_token_counts() to get cumulative counts across ALL steps
# Using model.get_token_counts() only returns the LAST API call's tokens!
token_counts = agent.monitor.get_total_token_counts()
total_input_tokens = token_counts.get('input', 0) or 0
total_output_tokens = token_counts.get('output', 0) or 0

print(f"\nTotal Input Tokens: {total_input_tokens}")
print(f"Total Output Tokens: {total_output_tokens}")



Total Input Tokens: 8190
Total Output Tokens: 414


In [175]:
# Display step details
print("Step Durations (in seconds):")
for i, duration in enumerate(agent.monitor.step_durations):
    print(f"  Step {i}: {duration:.2f}s")


Step Durations (in seconds):
  Step 0: 6.99s
  Step 1: 6.12s
  Step 2: 6.69s


In [176]:
results_df

,timestamp,model,question,total_steps,total_agent_time_s,notebook_runtime_s,input_tokens,output_tokens,answer,provider
0,2025-11-11T17:06:04.551948,x-ai/grok-4-fast,What are the unique set of merchants in the pa...,4,14.74,15.95,8942,2707,"Belles_cookbook_store,Crossfit_Hanna,Golfclub_...",NaN
1,2025-11-11T17:06:29.566669,x-ai/grok-4-fast,What are the unique set of merchants in the pa...,4,9.41,10.65,8517,1688,"Crossfit_Hanna,Belles_cookbook_store,Golfclub_...",NaN
2,2025-11-11T17:08:38.253162,x-ai/grok-4-fast,What are the unique set of merchants in the pa...,6,15.69,-6.02,16394,2659,"Belles_cookbook_store, Crossfit_Hanna, Golfclu...",NaN
3,2025-11-11T17:09:39.490712,x-ai/grok-4-fast,What are the unique set of merchants in the pa...,2,6.37,-12.64,2360,942,"Belles_cookbook_store,Crossfit_Hanna,Golfclub_...",NaN
4,2025-11-11T17:12:38.314130,x-ai/grok-4-fast,What are the unique set of merchants in the pa...,4,8.78,10.13,9187,1304,"Belles_cookbook_store, Crossfit_Hanna, Golfclu...",NaN
5,2025-11-11T17:13:35.201777,qwen/qwen3-coder,What are the unique set of merchants in the pa...,3,10.44,11.41,4891,128,"Crossfit_Hanna, Belles_cookbook_store, Golfclu...",NaN
6,2025-11-11T17:15:07.367432,qwen/qwen3-coder,What are the unique set of merchants in the pa...,3,12.35,13.43,4889,124,"Crossfit_Hanna, Belles_cookbook_store, Golfclu...",NaN
7,2025-11-11T17:16:58.018352,openai/gpt-5-mini,What are the unique set of merchants in the pa...,3,56.10,57.19,6032,3527,"Belles_cookbook_store, Crossfit_Hanna, Golfclu...",NaN
8,2025-11-11T17:18:05.009106,openai/gpt-oss-120b,What are the unique set of merchants in the pa...,4,10.83,11.90,7404,931,"Belles_cookbook_store, Crossfit_Hanna, Golfclu...",NaN
9,2025-11-11T17:19:08.738898,z-ai/glm-4.5-air,What are the unique set of merchants in the pa...,3,11.45,12.54,5602,357,"Belles_cookbook_store, Crossfit_Hanna, Golfclu...",NaN
